# Execução Dirigida — Transfer Learning (Experiência Guiada)

### Objetivo
Vivenciar, em poucos passos, o uso de **modelos pré-treinados** para perceber o princípio de **Transfer Learning**.

- **Máximo de 3 passos operacionais** e **1 decisão** (escolha de caminho).
- **Comprovação simples**: uma **saída** do código **+** um **relato breve** (3–5 linhas) no final do notebook.

---

### Como funciona
1. **Passo 1** – Preparar o ambiente e detectar o dispositivo (CPU/CUDA/MPS).  
2. **Passo 2 (decisão)** – Escolher **um** caminho:  
   - **(A) Visão**: carregar a **ResNet50** pré-treinada.  
   - **(B) Texto**: carregar o **GPT-2** e **gerar** uma resposta curta.  
3. **Passo 3** – Registrar a **saída** e escrever uma **reflexão** breve.

> **Observação**: O download dos pesos pré-treinados requer internet. Se estiver offline, execute assim mesmo para registrar a tentativa (o notebook captura a mensagem de erro).

## Passo 1 — Preparar o ambiente (dispositivo e versões)
Execute a célula para identificar o dispositivo e registrar versões principais (úteis na sua evidência).

In [ ]:
# Dispositivo (CUDA / MPS / CPU)
import torch, sys

device = torch.device("cuda" if torch.cuda.is_available()
                      else "mps" if hasattr(torch, "mps") and torch.mps.is_available()
                      else "cpu")
print("Dispositivo detectado:", device)

# Versões para registro
import transformers, datasets
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)

## Passo 2 — Escolha **um** caminho (A **ou** B)

### (A) Visão — ResNet50 como **extratora de características**
- Carrega a ResNet50 pré-treinada (ImageNet) e mostra a camada final.  
- **Saída esperada**: nome do modelo e dimensões da última camada.

> Execute **esta** célula se escolher **Visão**.


In [ ]:
# Caminho (A) Visão — ResNet50
try:
    import torchvision
    from torchvision import models
    from torch import nn

    # Carregar pesos pré-treinados (requer internet na 1ª vez)
    resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    for p in resnet.parameters():
        p.requires_grad = False  # usar como extratora de características

    n_features = resnet.fc.in_features
    print("Modelo:", type(resnet).__name__)
    print("Última camada (antes da substituição):", resnet.fc)
    print("Dimensões do extrator de features:", n_features)

    # Exemplo: substituir a cabeça para 3 classes (apenas demonstração)
    resnet.fc = nn.Linear(n_features, 3)
    print("Nova cabeça (Linear -> 3 classes):", resnet.fc)

    # (Opcional) Simular um passo de inferência com tensor aleatório (sem imagens reais)
    x = torch.randn(2, 3, 224, 224)  # 2 imagens simuladas 224x224 RGB
    with torch.no_grad():
        y = resnet(x)
    print("Saída simulada (batch=2, classes=3):", y.shape)

    evidencia_visao = f"Modelo: ResNet50 | features: {n_features} | saida_simulada: {tuple(y.shape)}"
except Exception as e:
    evidencia_visao = f"[Visão] Falha ao carregar/rodar ResNet50 -> {type(e).__name__}: {e}"
    print(evidencia_visao)

### (B) Texto — GPT-2 para **gerar** uma resposta curta
- Carrega GPT-2 pré-treinado e gera uma sequência curta.  
- **Saída esperada**: texto gerado pelo modelo.

> Execute **esta** célula se escolher **Texto**.


In [ ]:
# Caminho (B) Texto — GPT-2
evidencia_texto = None
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Seleção de dispositivo para o pipeline (CUDA > MPS > CPU)
    if torch.cuda.is_available():
        dev = 0
    elif hasattr(torch, "mps") and torch.mps.is_available():
        # algumas versões de transformers não aceitam "mps"; fallback para CPU
        try:
            dev = "mps"
        except Exception:
            dev = -1
    else:
        dev = -1

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=dev)

    prompt = "Explique o que é transferência de aprendizado em IA."
    out = pipe(prompt, max_new_tokens=40, temperature=0.8, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
    texto_gerado = out[0]["generated_text"]
    print("Texto gerado:\n", texto_gerado)

    evidencia_texto = "Texto gerado OK (GPT-2)"
except Exception as e:
    evidencia_texto = f"[Texto] Falha ao carregar/gerar com GPT-2 -> {type(e).__name__}: {e}"
    print(evidencia_texto)

## Passo 3 — Evidência + Relato breve (3–5 linhas)

- **Evidência**: uma saída do caminho escolhido (A **ou** B).  
- **Relato**: descreva brevemente o que observou sobre o **reaproveitamento** do conhecimento no modelo pré-treinado.

> Preencha e rode a célula abaixo.


In [ ]:
# Preencha seu relato aqui (entre aspas triplas):
relato = """
No meu teste, observei...
"""

# Coleta automática de evidência (pega qualquer uma disponível)
evidencia = None
for v in ["evidencia_visao", "evidencia_texto"]:
    if v in globals() and globals()[v]:
        evidencia = globals()[v]
        break

print("=== EVIDÊNCIA ===")
print(evidencia if evidencia else "Sem evidência capturada (ok se você estiver offline, descreva no relato).")
print("\n=== RELATO ===")
print(relato.strip())

---
### Entrega no Fórum

Para concluir esta atividade, poste uma única mensagem no fórum contendo:

#### 1.	Evidência da execução

Copie e cole a saída principal obtida no notebook (por exemplo, o texto gerado pelo GPT-2 ou o resumo do modelo carregado da ResNet50).

#### 2.	Relato breve (3–5 linhas)

Explique, com suas palavras:
- O que o modelo já sabia antes do seu teste?
- O que você observou na execução (resultado, comportamento ou erro)?
- O que aprendeu sobre o conceito de transferência de aprendizado?

**Importante**:

Se o ambiente não permitiu baixar ou rodar o modelo (por falta de internet, GPU etc.), registre o ocorrido e descreva o que entendeu sobre o processo.

Mesmo sem resultado numérico, isso vale como experiência válida — o essencial é demonstrar compreensão e reflexão sobre o funcionamento do Transfer Learning.